In [14]:
import pandas as pd
import re
import json

In [72]:
with open('jobs_data2.json', 'r') as f:
    data2 = json.load(f)
with open('jobs_data.json', 'r') as f:
    data = json.load(f)
    
data.extend(data2)

df = pd.json_normalize(data)

df_exploded = df.explode('skills').reset_index(drop=True)

print(df_exploded.info())
display(df_exploded.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170848 entries, 0 to 170847
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   post_id       170848 non-null  object
 1   company_name  170848 non-null  object
 2   job_name      170848 non-null  object
 3   skills        170801 non-null  object
 4   date_scraped  170848 non-null  object
 5   date_posted   170848 non-null  object
dtypes: object(6)
memory usage: 7.8+ MB
None


,post_id,company_name,job_name,skills,date_scraped,date_posted
0,GLOBINVA,"Global Infotek, Inc.",Senior Business Development Lead - BDWFH 1643,Security Clearance,2026-02-12,1 day ago
1,GLOBINVA,"Global Infotek, Inc.",Senior Business Development Lead - BDWFH 1643,DoD,2026-02-12,1 day ago
2,GLOBINVA,"Global Infotek, Inc.",Senior Business Development Lead - BDWFH 1643,Embedded Systems,2026-02-12,1 day ago
3,GLOBINVA,"Global Infotek, Inc.",Senior Business Development Lead - BDWFH 1643,.NET,2026-02-12,1 day ago
4,GLOBINVA,"Global Infotek, Inc.",Senior Business Development Lead - BDWFH 1643,WAR,2026-02-12,1 day ago


# 1. Handling Missing Values

In [73]:
df_exploded = df_exploded.drop_duplicates(keep='first')
df_exploded = df_exploded.dropna()


print(df_exploded.info())

<class 'pandas.core.frame.DataFrame'>
Index: 160924 entries, 0 to 170847
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   post_id       160924 non-null  object
 1   company_name  160924 non-null  object
 2   job_name      160924 non-null  object
 3   skills        160924 non-null  object
 4   date_scraped  160924 non-null  object
 5   date_posted   160924 non-null  object
dtypes: object(6)
memory usage: 8.6+ MB
None


# 2. Data Type Conversion

In [74]:
df_exploded['post_id'] = df_exploded['post_id'].astype("string")
df_exploded['company_name'] = df_exploded['company_name'].astype("string")
df_exploded['job_name'] = df_exploded['job_name'].astype("string")
df_exploded['skills'] = df_exploded['skills'].astype("string")
df_exploded['date_scraped'] = pd.to_datetime(df_exploded['date_scraped'], errors='coerce')
df_exploded['date_posted'] = df_exploded['date_posted'].astype("string")

df_exploded['date_scraped'] = pd.to_datetime(df_exploded['date_scraped'])

# 2. ฟังก์ชันสำหรับสกัดตัวเลขจาก "x days ago" แล้วลบออกจากวันที่ scrape
def convert_relative_date(row):
    text = str(row['date_posted']).lower()
    
    # ใช้ Regex หาตัวเลขในข้อความ
    days_match = re.search(r'(\d+)', text)
    
    if 'day' in text and days_match:
        days_to_subtract = int(days_match.group(1))
        return row['date_scraped'] - pd.Timedelta(days=days_to_subtract)
    
    elif 'today' in text or 'hour' in text:
        return row['date_scraped']
    
    return pd.NaT # ถ้าไม่เข้าเงื่อนไขให้คืนค่าเป็น Not a Time

# 3. Apply เพื่อสร้างคอลัมน์ใหม่
df_exploded['actual_date_posted'] = df_exploded.apply(convert_relative_date, axis=1)

print(df_exploded.info())
display(df_exploded.head())

<class 'pandas.core.frame.DataFrame'>
Index: 160924 entries, 0 to 170847
Data columns (total 7 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   post_id             160924 non-null  string        
 1   company_name        160924 non-null  string        
 2   job_name            160924 non-null  string        
 3   skills              160924 non-null  string        
 4   date_scraped        160924 non-null  datetime64[ns]
 5   date_posted         160924 non-null  string        
 6   actual_date_posted  160924 non-null  datetime64[ns]
dtypes: datetime64[ns](2), string(5)
memory usage: 9.8 MB
None


,post_id,company_name,job_name,skills,date_scraped,date_posted,actual_date_posted
0,GLOBINVA,"Global Infotek, Inc.",Senior Business Development Lead - BDWFH 1643,Security Clearance,2026-02-12,1 day ago,2026-02-11
1,GLOBINVA,"Global Infotek, Inc.",Senior Business Development Lead - BDWFH 1643,DoD,2026-02-12,1 day ago,2026-02-11
2,GLOBINVA,"Global Infotek, Inc.",Senior Business Development Lead - BDWFH 1643,Embedded Systems,2026-02-12,1 day ago,2026-02-11
3,GLOBINVA,"Global Infotek, Inc.",Senior Business Development Lead - BDWFH 1643,.NET,2026-02-12,1 day ago,2026-02-11
4,GLOBINVA,"Global Infotek, Inc.",Senior Business Development Lead - BDWFH 1643,WAR,2026-02-12,1 day ago,2026-02-11


# 3. Handling Inconsistency

In [85]:
import re
import pandas as pd

# 1. สร้าง Mapping Dictionary แบบละเอียด (จัดหมวดหมู่และอ้างอิงภาษา)
tech_mapping = {
    # ---------------- Programming Languages ----------------
    'python': {'category': 'Programming Language', 'language': 'Python'},
    'sql': {'category': 'Programming Language', 'language': 'SQL'},
    'sas': {'category': 'Programming Language', 'language': 'SAS'},
    'r': {'category': 'Programming Language', 'language': 'R'},
    'java': {'category': 'Programming Language', 'language': 'Java'},
    'kotlin': {'category': 'Programming Language', 'language': 'Kotlin'},
    'c++': {'category': 'Programming Language', 'language': 'C++'},
    'c#': {'category': 'Programming Language', 'language': 'C#'},
    'javascript': {'category': 'Programming Language', 'language': 'JavaScript/TypeScript'},
    'typescript': {'category': 'Programming Language', 'language': 'JavaScript/TypeScript'},
    'go': {'category': 'Programming Language', 'language': 'Go'},
    'scala': {'category': 'Programming Language', 'language': 'Scala'},
    'php': {'category': 'Programming Language', 'language': 'PHP'},
    'ruby': {'category': 'Programming Language', 'language': 'Ruby'},
    'swift': {'category': 'Programming Language', 'language': 'Swift'},
    'rust': {'category': 'Programming Language', 'language': 'Rust'},
    'html': {'category': 'Programming Language', 'language': 'HTML'},
    'css': {'category': 'Programming Language', 'language': 'CSS'},
    'bash': {'category': 'Programming Language', 'language': 'Bash & Shell'},
    'shell': {'category': 'Programming Language', 'language': 'Bash & Shell'},
    'powershell': {'category': 'Programming Language', 'language': 'PowerShell'},
    'perl': {'category': 'Programming Language', 'language': 'Perl'},
    
    # ---------------- Frameworks & Libraries ----------------
    # Python
    'pandas': {'category': 'Framework & Library', 'language': 'Python'},
    'numpy': {'category': 'Framework & Library', 'language': 'Python'},
    'scikit-learn': {'category': 'Framework & Library', 'language': 'Python'},
    'tensorflow': {'category': 'Framework & Library', 'language': 'Python'},
    'pytorch': {'category': 'Framework & Library', 'language': 'Python'},
    'django': {'category': 'Framework & Library', 'language': 'Python'},
    'flask': {'category': 'Framework & Library', 'language': 'Python'},
    'fastapi': {'category': 'Framework & Library', 'language': 'Python'},
    'pyspark': {'category': 'Framework & Library', 'language': 'Python'},
    'selenium': {'category': 'Framework & Library', 'language': 'Python/Java'},
    
    # Java & Scala
    'spring boot': {'category': 'Framework & Library', 'language': 'Java'},
    'spring': {'category': 'Framework & Library', 'language': 'Java'},
    'hibernate': {'category': 'Framework & Library', 'language': 'Java'},
    'junit': {'category': 'Framework & Library', 'language': 'Java'},
    'kafka': {'category': 'Framework & Library', 'language': 'Java/Scala'},
    'spark': {'category': 'Framework & Library', 'language': 'Scala'},
    'akka': {'category': 'Framework & Library', 'language': 'Scala'},
    
    # JavaScript/TypeScript
    'react': {'category': 'Framework & Library', 'language': 'JavaScript/TypeScript'},
    'node.js': {'category': 'Framework & Library', 'language': 'JavaScript/TypeScript'},
    'angular': {'category': 'Framework & Library', 'language': 'JavaScript/TypeScript'},
    'vue': {'category': 'Framework & Library', 'language': 'JavaScript/TypeScript'},
    'react native': {'category': 'Framework & Library', 'language': 'JavaScript/TypeScript'},
    
    # C# & C++
    'net core': {'category': 'Framework & Library', 'language': 'C#'},
    'entity framework': {'category': 'Framework & Library', 'language': 'C#'},
    'unity': {'category': 'Framework & Library', 'language': 'C#'},
    'boost': {'category': 'Framework & Library', 'language': 'C++'},
    
    # PHP & CSS
    'wordpress': {'category': 'Framework & Library', 'language': 'PHP'},
    'drupal': {'category': 'Framework & Library', 'language': 'PHP'},
    'magento': {'category': 'Framework & Library', 'language': 'PHP'},
    'sass': {'category': 'Framework & Library', 'language': 'CSS'},
    'bootstrap': {'category': 'Framework & Library', 'language': 'CSS'},

    # ---------------- Cloud, Infra & DevOps ----------------
    'aws': {'category': 'Cloud & Infra', 'language': 'None'},
    'amazon web services': {'category': 'Cloud & Infra', 'language': 'None'}, # เพิ่มชื่อเต็ม
    'azure': {'category': 'Cloud & Infra', 'language': 'None'},
    'microsoft azure': {'category': 'Cloud & Infra', 'language': 'None'}, # เพิ่มชื่อเต็ม
    'gcp': {'category': 'Cloud & Infra', 'language': 'None'},
    'google cloud platform': {'category': 'Cloud & Infra', 'language': 'None'}, # เพิ่มชื่อเต็ม
    'docker': {'category': 'Cloud & Infra', 'language': 'None'},
    'kubernetes': {'category': 'Cloud & Infra', 'language': 'None'},
    'terraform': {'category': 'Cloud & Infra', 'language': 'None'},
    'snowflake': {'category': 'Cloud & Infra', 'language': 'SQL'},
    'databricks': {'category': 'Cloud & Infra', 'language': 'None'},
    'linux': {'category': 'Cloud & Infra', 'language': 'None'},
    'unix': {'category': 'Cloud & Infra', 'language': 'None'},
    'ci/cd': {'category': 'Cloud & Infra', 'language': 'None'},
    'jenkins': {'category': 'Cloud & Infra', 'language': 'None'},
    'git': {'category': 'Cloud & Infra', 'language': 'None'},
    'github': {'category': 'Cloud & Infra', 'language': 'None'},
    'gitlab': {'category': 'Cloud & Infra', 'language': 'None'},
    'bitbucket': {'category': 'Cloud & Infra', 'language': 'None'},
    'ansible': {'category': 'Cloud & Infra', 'language': 'None'},
    'puppet': {'category': 'Cloud & Infra', 'language': 'None'},
    
    # ---------------- Data Visualization & BI Tools ----------------
    'tableau': {'category': 'Data Viz & BI', 'language': 'None'},
    'power bi': {'category': 'Data Viz & BI', 'language': 'None'},
    'alteryx': {'category': 'Data Viz & BI', 'language': 'None'},
    'looker': {'category': 'Data Viz & BI', 'language': 'None'},
    'metabase': {'category': 'Data Viz & BI', 'language': 'None'},
    
    # ---------------- Database ----------------
    'postgresql': {'category': 'Database', 'language': 'SQL'},
    'mysql': {'category': 'Database', 'language': 'SQL'},
    'oracle': {'category': 'Database', 'language': 'SQL'},
    'redshift': {'category': 'Database', 'language': 'SQL'},
    'bigquery': {'category': 'Database', 'language': 'SQL'},
    'sqlite': {'category': 'Database', 'language': 'SQL'},
    'mongodb': {'category': 'Database', 'language': 'NoSQL'},
    'redis': {'category': 'Database', 'language': 'NoSQL'},
    'dynamodb': {'category': 'Database', 'language': 'NoSQL'},
    'cassandra': {'category': 'Database', 'language': 'NoSQL'},
    'neo4j': {'category': 'Database', 'language': 'NoSQL'},
    'nosql': {'category': 'Database', 'language': 'NoSQL'},
}

def extract_tech_details(skill):
    if not isinstance(skill, str):
        return []
    
    skill_clean = skill.lower().replace('/', ' ')
    found_items = []
    
    for tech, details in tech_mapping.items():
        # เช็คด้วย regex เพื่อให้เจอเฉพาะคำนั้นๆ จริงๆ
        if re.search(r'\b' + re.escape(tech) + r'\b', skill_clean):
            # สร้าง Dictionary สำหรับแต่ละ Tech ที่เจอ
            found_items.append({
                'tech_name': tech.upper(),
                'tech_category': details['category'],
                'base_language': details['language']
            })
            
    # เราไม่สามารถ set() dictionary ได้ตรงๆ จึงต้องใช้วิธีนำ dictionary ออกมาเรียงไม่ให้ซ้ำกัน
    unique_items = {item['tech_name']: item for item in found_items}.values()
    return list(unique_items)

# 1. Apply สร้างคอลัมน์ที่เป็น List ของ Dictionaries
df_exploded['tech_details'] = df_exploded['skills'].apply(extract_tech_details)

# 2. แตก List ออกมาเป็นแต่ละ Row
df_cleaned = df_exploded.explode('tech_details')
df_cleaned = df_cleaned.dropna(subset=['tech_details'])

# 3. แตก Dictionary ในคอลัมน์ใหม่ออกมาเป็นคอลัมน์แยกๆ (สำคัญมาก)
# สร้าง DataFrame จากคอลัมน์ dictionary แล้วนำมาประกอบกลับร่าง (Concat)
tech_df = df_cleaned['tech_details'].apply(pd.Series)
df_final = pd.concat([df_cleaned.drop('tech_details', axis=1), tech_df], axis=1).reset_index(drop=True)

# 1. สร้างฟังก์ชันสำหรับแยกคำ โดยดัก N/A เอาไว้
def split_language(lang):
    lang_str = str(lang).strip()
    if lang_str == 'N/A':
        return [lang_str] # ถ้าเป็น N/A ให้ส่งกลับไปแบบเดิม ไม่ต้องตัด
    else:
        # ถ้าไม่ใช่ N/A ให้ตัดด้วย '/' และลบช่องว่างหน้า-หลังให้เรียบร้อย
        return [l.strip() for l in lang_str.split('/')]

# 2. นำฟังก์ชันไปประยุกต์ใช้กับคอลัมน์ base_language
df_final['base_language'] = df_final['base_language'].apply(split_language)

# 3. แตก List ออกเป็นแถวใหม่
df_final = df_final.explode('base_language')

print(df_final.info())
display(df_final.head())

<class 'pandas.core.frame.DataFrame'>
Index: 12519 entries, 0 to 11526
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   post_id             12519 non-null  string        
 1   company_name        12519 non-null  string        
 2   job_name            12519 non-null  string        
 3   skills              12519 non-null  string        
 4   date_scraped        12519 non-null  datetime64[ns]
 5   date_posted         12519 non-null  string        
 6   actual_date_posted  12519 non-null  datetime64[ns]
 7   tech_name           12519 non-null  object        
 8   tech_category       12519 non-null  object        
 9   base_language       12519 non-null  object        
dtypes: datetime64[ns](2), object(3), string(5)
memory usage: 1.1+ MB
None


,post_id,company_name,job_name,skills,date_scraped,date_posted,actual_date_posted,tech_name,tech_category,base_language
0,GLOBINVA,"Global Infotek, Inc.",Lead Engineer - Legacy Software Baseline - POK...,GitLab,2026-02-12,1 day ago,2026-02-11,GITLAB,Cloud & Infra,None
1,GLOBINVA,"Global Infotek, Inc.",Lead Engineer - Legacy Software Baseline - POK...,Amazon Web Services,2026-02-12,1 day ago,2026-02-11,AMAZON WEB SERVICES,Cloud & Infra,None
2,GLOBINVA,"Global Infotek, Inc.",Lead Engineer - Legacy Software Baseline - POK...,Microsoft Azure,2026-02-12,1 day ago,2026-02-11,AZURE,Cloud & Infra,None
3,GLOBINVA,"Global Infotek, Inc.",Lead Engineer - Legacy Software Baseline - POK...,Microsoft Azure,2026-02-12,1 day ago,2026-02-11,MICROSOFT AZURE,Cloud & Infra,None
4,GLOBINVA,"Global Infotek, Inc.",Cloud Engineer - Lead BBPO 1642,Google Cloud Platform,2026-02-12,1 day ago,2026-02-11,GOOGLE CLOUD PLATFORM,Cloud & Infra,None


บันทึก File JSON

In [86]:
df_final.to_json('cleaned_jobs_data.json', orient='records', indent=4,date_format='iso')